In [242]:
import datetime
import pandas as pd
with open("../.env") as fo:
    github_token = fo.read()[:-1]
session_start = datetime.datetime.utcnow()
print(session_start)

2019-07-22 21:14:09.714877


## Github API v4 GraphQL 
Using the requests module as suggested in the top answer at [How to consume the Github GraphQL API using Python?](https://stackoverflow.com/questions/45957784/how-to-consume-the-github-graphql-api-using-python)

In [249]:
import requests
github_url = 'https://api.github.com/graphql'
github_headers = {'Authorization': f"token {github_token}"}
query = {
    'query': """{
        repository(owner:"LD4P", name:"sinopia_editor") {
          issues(states:OPEN, first:100) {
            edges {
              node {
                title
                url
                number
              }
            }
          } 
       }
    }
    """
}
result = requests.post(url=github_url, json=query, headers=github_headers)

In [253]:
def issues_query(state='OPEN', first=100):
    query = {
        'query': """{{
            repository(owner:"LD4P", name:"sinopia_editor") {{
              issues(first:{first}) {{
                pageInfo {{
                    startCursor
                    hasNextPage
                    endCursor
                }}
                edges {{
                  node {{
                    title
                    url
                    number
                    createdAt
                    closedAt
                    projectCards(first:5) {{
                      edges {{
                          node {{
                              project {{
                                      name
                                      number
                              }}
                              state
                          }}
                      }}
                    }}
                    participants(first:10) {{
                        edges {{
                            node {{
                                login
                            }}
                        }}
                    }}
                    state
                  }}
                }}
              }} 
           }}
        }}
    """.format(state=state, first=first)
    }
    result = requests.post(url=github_url, json=query, headers=github_headers)
    return result.json()
    
def commits_query(repo):
    query = {
        "query": """{{
            repository(owner:"LD4P", name:"{repo}") {{
                refs(refPrefix: "refs/master/"){{
                    totalCount
                }}
        
            }}
        }}
        """.format(repo=repo)
    }
    result = requests.post(url=github_url, json=query, headers=github_headers)
    return result.json()


In [254]:
first_issues = issues_query()

In [255]:
first_issues

{'data': {'repository': {'issues': {'pageInfo': {'startCursor': 'Y3Vyc29yOnYyOpHOFbQ6kg==',
     'hasNextPage': True,
     'endCursor': 'Y3Vyc29yOnYyOpHOFrpukA=='},
    'edges': [{'node': {'title': 'testing suite for sinopia_editor ',
       'url': 'https://github.com/LD4P/sinopia_editor/issues/1',
       'number': 1,
       'createdAt': '2018-09-26T17:24:27Z',
       'closedAt': '2018-09-27T22:55:09Z',
       'projectCards': {'edges': []},
       'participants': {'edges': [{'node': {'login': 'ndushay'}},
         {'node': {'login': 'mejackreed'}},
         {'node': {'login': 'jermnelson'}}]},
       'state': 'CLOSED'}},
     {'node': {'title': 'Code syntax/style checking w ESLint for sinopia_editor',
       'url': 'https://github.com/LD4P/sinopia_editor/issues/2',
       'number': 2,
       'createdAt': '2018-09-26T17:30:33Z',
       'closedAt': '2018-09-26T23:01:05Z',
       'projectCards': {'edges': []},
       'participants': {'edges': [{'node': {'login': 'ndushay'}},
         {'no

In [221]:
result = commits_query("sinopia_editor")
print(result)

{'data': {'repository': {'refs': {'totalCount': 0}}}}


In [224]:
import os, sys

In [235]:
os.path.abspath('.')
sys.path.append('/Users/jpnelson/2019/project-notebooks/')
import src.sinopia.github as sinopia

FileNotFoundError: [Errno 2] No such file or directory: '../../.env'

In [232]:
dir(sinopia)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

## Github API v3
Using the [PyGithub](https://pygithub.readthedocs.io/en/latest/) package for metrics generation on the Sinopia project. current focus is on the `sinopia_editor` repository.

In [243]:
from github import Github

g = Github(github_token)

sinopia_editor = g.get_repo("LD4P/sinopia_editor")
open_issues = sinopia_editor.get_issues(state='open')
closed_issues = sinopia_editor.get_issues(state='closed')

In [244]:
print(f"## Session At {session_start}\nopen issues: {open_issues.totalCount}, closed: {closed_issues.totalCount}")

## Session At 2019-07-22 21:14:09.714877
open issues: 83, closed: 942


In [245]:
closed_issues[0].created_at

datetime.datetime(2019, 7, 19, 20, 15, 41)

In [99]:
def v3_closed_issues_date_frame(repo):
    closed_issues = repo.get_issues(state='closed')
    created_series, closed_series = {}, {}
    for row in closed_issues:
        created_series[row.number] =  row.created_at
        closed_series[row.number] = row.closed_at
    date_data_frame = pd.DataFrame({ "created": created_series,
                                  "closed": closed_series })
    date_data_frame["elasped"] = date_data_frame["closed"] -  date_data_frame["created"]
    return date_data_frame

def v3_filter_issues(repo, func=closed_issues_date_frame, since=None):
    data_frame = func(repo)
    if since is None:
        closed_issues = data_frame
        since = closed_issues[0]['created']
    else:
        start = pd.Timestamp(since)
        filter_mask = data_frame['created'] > start
        closed_issues = data_frame[filter_mask]
    return closed_issues

def v4_closed_issues_report(data_frame):
    since = data_frame['created'][data_frame['created'].index[0]]
    print(f"## Statistics on Closed Issues from {since}")
    print(f"- **Total**: {len(data_frame)}")
    print(f"- **Mean**: {data_frame['elasped'].mean()}")
    print(f"- **Median**: {data_frame['elasped'].median()}"), 
    print(f"- **Maxiumum number of elasped time**: {date_data_frame['elasped'].max()}")
    print(f"- **Fastest elasped time**: {date_data_frame['elasped'].min()}")

In [110]:
def v4_open_issues_data_frame(repo):
    open_issues = repo.get_issues(state='open')
    titles, created, state = {}, {}, {}
    for issue in open_issues:
        titles[issue.number] = issue.title
        created[issue.number] = issue.created_at
        state[issue.number] = issue.state
    data_frame = pd.DataFrame({ "titles": titles, "created": created, "state": state})
    return data_frame

## Session At 2019-07-19 17:51:21.177430
open issues: 77, closed: 935

## Session At 2019-07-18 16:26:17.849673
open issues: 81, closed: 920

In [105]:
sprint_closed = filter_issues(sinopia_editor, since=datetime.datetime(2019,7,3))

In [109]:
closed_issues[0].state

'closed'

In [113]:
sprint_open = filter_issues(sinopia_editor, func=open_issues_data_frame, since=datetime.datetime(2019,7,3))

In [118]:
dir(open_issues[3])

['CHECK_AFTER_INIT_FLAG',
 '_CompletableGithubObject__complete',
 '_CompletableGithubObject__completed',
 '_GithubObject__makeSimpleAttribute',
 '_GithubObject__makeSimpleListAttribute',
 '_GithubObject__makeTransformedAttribute',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_assignee',
 '_assignees',
 '_body',
 '_closed_at',
 '_closed_by',
 '_comments',
 '_comments_url',
 '_completeIfNeeded',
 '_completeIfNotSet',
 '_created_at',
 '_events_url',
 '_headers',
 '_html_url',
 '_id',
 '_identity',
 '_initAttributes',
 '_labels',
 '_labels_url',
 '_makeBoolAttribute',
 '_makeClassAttribute',
 '_makeDatetimeAttribute',
 '_makeDictAttribute',
 '_makeDictOfStringsToClasse

## Current Session At 2019-07-18 16:26:17.849673
open issues: 83, closed: 918


In [ ]:
sprint_closed_df = closed_issues(sinopia_editor, since=datetime.datetime(2019, 7, 3))

## Current Session 2019-07-18 16:26:17.849673

At 2019-07-18 16:26:17.849673
        
open issues: 83, closed: 916

## Statistics on Closed Issues from 2019-07-03 01:04:09
- **Total**: 115
- **Mean**: 1 days 08:00:54.843478
- **Median**: 0 days 01:33:34
- **Maxiumum number of elasped time**: 335 days 15:05:14
- **Fastest elasped time**: 0 days 00:00:24

In [47]:
sprint_closed_df = closed_issues(sinopia_editor, since=datetime.datetime(2019, 7, 3))

In [59]:
closed_issues_report(sprint_closed_df)

## Statistics on Closed Issues from 2019-07-03 01:04:09
- **Total**: 115
- **Mean**: 1 days 08:00:54.843478
- **Median**: 0 days 01:33:34
- **Maxiumum number of elasped time**: 335 days 15:05:14
- **Fastest elasped time**: 0 days 00:00:24


In [68]:
for row in sprint_closed_df['elasped'].index:
    print(f"{row} {sprint_closed_df['elasped'][row]}")

871 9 days 14:43:07
872 4 days 23:05:29
874 14 days 03:10:38
875 0 days 00:45:00
876 0 days 00:55:29
877 0 days 00:20:49
878 0 days 00:24:30
879 9 days 01:39:05
880 0 days 00:27:47
881 8 days 22:16:52
882 1 days 21:20:42
884 14 days 21:08:19
885 4 days 21:48:30
887 0 days 00:05:54
888 14 days 18:48:15
889 8 days 16:53:19
890 0 days 12:50:18
891 6 days 23:04:32
892 0 days 00:05:12
893 2 days 14:24:32
894 3 days 09:20:19
895 0 days 00:25:30
896 3 days 19:05:21
897 0 days 02:21:03
898 2 days 23:34:46
899 0 days 15:57:09
900 2 days 23:57:54
902 0 days 00:11:59
903 0 days 00:43:51
904 0 days 00:04:56
905 0 days 00:08:55
906 0 days 00:08:11
907 0 days 00:46:15
908 0 days 11:49:04
909 0 days 15:19:22
910 0 days 12:57:05
911 0 days 01:39:37
912 0 days 01:30:54
913 0 days 01:27:03
914 0 days 00:18:35
915 0 days 01:36:04
916 1 days 19:46:35
917 0 days 00:03:27
918 0 days 00:16:37
919 0 days 00:03:31
920 0 days 03:28:28
921 0 days 03:13:08
922 0 days 00:00:24
923 0 days 01:17:54
924 0 days 00:56:

In [81]:
open_df = filter_issues(sinopia_editor, func=open_issues_data_frame, since=datetime.datetime(2019, 7, 3))

In [92]:
len(open_df)

16

## Current Session 2019-07-17 18:25:09.812405

At 2019-07-17 18:25:09.812405

open issues: 82, closed: 895


In [10]:
created_series = {}
closed_series = {}
for row in closed_issues:
    created_series[row.number] =  row.created_at
    closed_series[row.number] = row.closed_at



In [11]:
date_data_frame = pd.DataFrame({ "created": created_series,
                                  "closed": closed_series })
date_data_frame["elasped"] = date_data_frame["closed"] -  date_data_frame["created"]

In [12]:
date_data_frame

,created,closed,elasped
1,2018-09-26 17:24:27,2018-09-27 22:55:09,1 days 05:30:42
2,2018-09-26 17:30:33,2018-09-26 23:01:05,0 days 05:30:32
3,2018-09-26 17:34:42,2018-09-28 14:59:47,1 days 21:25:05
4,2018-09-26 17:35:59,2019-03-12 16:39:19,166 days 23:03:20
6,2018-09-26 17:40:50,2018-11-20 21:35:29,55 days 03:54:39
7,2018-09-26 17:42:41,2019-03-13 22:12:29,168 days 04:29:48
8,2018-09-26 17:44:00,2019-03-13 22:12:52,168 days 04:28:52
9,2018-09-26 17:48:17,2018-09-26 22:14:43,0 days 04:26:26
10,2018-09-26 17:50:32,2018-09-26 19:20:21,0 days 01:29:49
11,2018-09-26 21:09:20,2018-09-26 22:14:43,0 days 01:05:23


## Statistics on Closed Issues at 2019-07-17 18:25:09.812405
- **Mean**: 15 days 16:19:16.442458
- **Median**: 0 days 21:37:39
- **Maxiumum number of elasped time**: 335 days 15:05:14
- **Fastest elasped time**: 0 days 00:00:24


## Statistics on Closed Issues at 2019-07-17 18:25:09.812405
- **Mean**: 15 days 16:19:16.442458
- **Median**: 0 days 21:37:39
- **Maxiumum number of elasped time**: 335 days 15:05:14
- **Fastest elasped time**: 0 days 00:00:24

In [15]:
all_commits = sinopia_editor.get_commits()

In [16]:
print(all_commits.totalCount)

1946


In [17]:
status_series, committer_series, last_modified_series = {},{},{}
for row in all_commits:
    status_series[row.sha] = row.get_combined_status().state
    committer_series[row.sha] = row.committer.login
    last_modified_series[row.sha] = row.last_modified

ReadTimeout: HTTPSConnectionPool(host='api.github.com', port=443): Read timed out. (read timeout=15)

In [36]:
print(dir(commits[2]))

['CHECK_AFTER_INIT_FLAG', '_CompletableGithubObject__complete', '_CompletableGithubObject__completed', '_GithubObject__makeSimpleAttribute', '_GithubObject__makeSimpleListAttribute', '_GithubObject__makeTransformedAttribute', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_author', '_comments_url', '_commit', '_committer', '_completeIfNeeded', '_completeIfNotSet', '_files', '_headers', '_html_url', '_identity', '_initAttributes', '_makeBoolAttribute', '_makeClassAttribute', '_makeDatetimeAttribute', '_makeDictAttribute', '_makeDictOfStringsToClassesAttribute', '_makeIntAttribute', '_makeListOfClassesAttribute', '_makeListOfDictsAttribute', '_makeListOfIntsAttribute', '_makeListOfListOfStringsA

In [40]:
for row in commits[2].get_statuses():
    print(row)

CommitStatus(state="success", id=7225135587, context="coverage/coveralls")
CommitStatus(state="success", id=7225129311, context="ci/circleci: build")
CommitStatus(state="pending", id=7225104844, context="ci/circleci: build")
CommitStatus(state="pending", id=7225104214, context="ci/circleci: build")
CommitStatus(state="success", id=7225104191, context="ci/circleci: dependencies")
CommitStatus(state="pending", id=7225091431, context="ci/circleci: dependencies")
CommitStatus(state="pending", id=7225089498, context="ci/circleci: dependencies")


In [44]:
commits[2].get_combined_status().state

'success'

In [45]:
commits[2].author.login

'jcoyne'

In [48]:
commits[2].sha

'a525cae6cd8006ddccb08966ecdcc445ab527422'

In [50]:
commits[2].parents[0].committer

NamedUser(login="web-flow")

In [56]:
commits[-3].committer

NamedUser(login="kefo")

In [240]:
closed_issues.keys()

dict_keys(['data'])

In [241]:
closed_issues['data']['repository']

dict_keys(['repository'])